#### Model taken from:
- [1] Offshore Pipelaying Dynamics. Gullik Anthon Jensen 
- [2] A nonlinear PDE formulation for offshore vessel pipeline installation. Gullik A. Jensen et al 
- [3] Modeling and Control of Offshore Pipelay Operations Based on a Finite Strain Pipe Model. Gullik A. Jensen 

In [1]:
import numpy as np
import numdifftools as nd
import math

In [2]:
mp = 96 #  (submerged) [kg/m]

In [3]:
diag_Irho = 1e12*np.array([1, 1, 2]) # [m^4]  , p.99 in [1]
 
Irho=np.diag(diag_Irho)

In [4]:
Irho

array([[1.e+12, 0.e+00, 0.e+00],
       [0.e+00, 1.e+12, 0.e+00],
       [0.e+00, 0.e+00, 2.e+12]])

In [5]:
qw = 1025 # Water density [kg/m3]
d0 = 0.761 # Outer diameter of pipe, [m]

In [6]:
diag_DT = 1.5*np.array([1, 1, 1]) # [N/m]  , p.99 in [1]

In [7]:
DT=np.diag(diag_DT) # (35) in [2]

In [8]:
DT

array([[1.5, 0. , 0. ],
       [0. , 1.5, 0. ],
       [0. , 0. , 1.5]])

In [9]:
from sympy import *

In [10]:
Dphi1, Dphi2, Dphi3 = symbols("Dphi1 Dphi2 Dphi3")

In [11]:
C1=np.array([Dphi1**2,
          (Dphi2**2+Dphi3**2)**0.5*Dphi2,
          (Dphi2**2+Dphi3**2)**0.5*Dphi3]).T

In [12]:
fD=1/2*d0*qw*np.dot(DT,C1)

In [13]:
Array(fD)

[585.01875*Dphi1**2, 585.01875*Dphi2*(Dphi2**2 + Dphi3**2)**0.5, 585.01875*Dphi3*(Dphi2**2 + Dphi3**2)**0.5]

In [14]:
# https://docs.scipy.org/doc/scipy-1.8.0/tutorial/integrate.html  
# 1-D Gray-Scott 

In [15]:
# Mat=np.matrix([[1.3,.4,6.7],[9,7,6],[3.3,5.6,7]])

In [16]:
I=np.identity(3)

In [17]:
fD

array([585.01875*Dphi1**2, 585.01875*Dphi2*(Dphi2**2 + Dphi3**2)**0.5,
       585.01875*Dphi3*(Dphi2**2 + Dphi3**2)**0.5], dtype=object)

In [18]:
np.dot(I,fD)

array([585.01875*Dphi1**2, 585.01875*Dphi2*(Dphi2**2 + Dphi3**2)**0.5,
       585.01875*Dphi3*(Dphi2**2 + Dphi3**2)**0.5], dtype=object)

In [19]:
Rb_t=I # https://www.researchgate.net/profile/Thor-Fossen/publication/224560837_Modeling_and_Control_of_Offshore_Pipelay_Operations_Based_on_a_Finite_Strain_Pipe_Model/links/00b7d520e175a3f918000000/Modeling-and-Control-of-Offshore-Pipelay-Operations-Based-on-a-Finite-Strain-Pipe-Model.pdf

In [20]:
Rb_t

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [21]:
diag_DR = 1.5*np.array([1, 1, 1]) #   p.4721 in [3]

In [22]:
DR=np.diag(diag_DR) 

In [23]:
dI= 0.69# Inner diameter of pipe, [m]

In [24]:
A=math.pi*((d0/2)**2-(dI/2)**2)

In [25]:
fg_e=np.array([0,0,(mp-qw*A)*9.81])

In [26]:
from numpy import linalg as LA

In [27]:
def k_sigma(hi,d0,fg_e):
    if hi<0:
        return 0
    elif 0<=hi<=d0/20:
        return LA.norm(fg_e,2)/(d0/8-d0/40)*10*hi**2/d0
    else:
        return LA.norm(fg_e,2)/(d0/8-d0/40)*(hi-d0/40)

In [28]:
def hi(phi,d0):
    return np.dot(phi.T,[0,0,1])+d0/2 

In [29]:
phi_test=np.array([2.3,3.5,7])

In [30]:
sigma= k_sigma(hi(phi_test,d0),d0,fg_e)*np.array([0,0,1])

In [31]:
sigma

array([    0.        ,     0.        , 12397.89846926])

In [32]:
# vessel motion
# Fossen book p.384

In [33]:
def psi(gamma,omega,Ct,Cr):
    coord=np.concatenate((gamma, omega), axis=0)
    zero=np.zeros((3,3))
    M1=np.concatenate((Ct, zero), axis=0)
    M2=np.concatenate((zero, Cr), axis=0)
    M=np.concatenate((M1, M2), axis=1)
    return 1/2*np.dot(np.dot(coord.T,M),coord)

In [34]:
from sympy import *

In [35]:
gamma1, gamma2,gamma3 = symbols("gamma1 gamma2 gamma3")
omega1, omega2,omega3 = symbols("omega1 omega2 omega3")

In [36]:
gamma=np.array([gamma1, gamma2,gamma3])
omega=np.array([omega1, omega2,omega3])

In [37]:
Ct=np.matrix([[1,0,0],[0,2,0],[0,0,3]])
Cr=np.matrix([[4,0,0],[0,5,0],[0,0,6]])

In [38]:
psi(gamma,omega,Ct,Cr)

matrix([[0.5*gamma1**2 + 1.0*gamma2**2 + 1.5*gamma3**2 + 2.0*omega1**2 + 2.5*omega2**2 + 3.0*omega3**2]],
       dtype=object)

In [39]:
fun = lambda x, y: psi(x,y,Ct,Cr)

In [40]:
Dfun = nd.Gradient(fun)

In [41]:
Dfun()

TypeError: __call__() missing 1 required positional argument: 'x'

In [ ]:
# coord=np.concatenate((gamma, omega), axis=0)

In [ ]:
gamma

In [ ]:
omega

In [ ]:
Dfun(gamma,omega)

In [ ]:
Dfun